# CS4048 - Lecture 21

## Multiple Linear Regression


Reference: Data 100, Fall 2025

[Acknowledgments Page](https://ds100.org/fa25/acks/)

In [ ]:
import pandas as pd
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
import matplotlib.pyplot as plt
import numpy as np

# Set display options for pandas and numpy
pd.options.mode.chained_assignment = None  # default='warn'
np.set_printoptions(suppress=True, precision=6)

<br/><br/>
<hr style="border: 5px solid #003262;" />
<hr style="border: 1px solid #fdb515;" />

## Fiting our Multiple Linear Regression Model

Consider the `penguins` dataset.

In [ ]:
# Load in the penguins dataset provided by seaborn package
df = sns.load_dataset("penguins")

# Filter to only include Adelie species and remove any rows with missing values.
# Remember that dropping rows with missing values should be done CAREFULLY.
# Here, we drop rows just so the demo modeling code runs without error.
df = df[df["species"] == "Adelie"].dropna()

df

Suppose we could measure flippers and weight easily, but not bill dimensions.
How can we predict **bill depth** from flipper length and/or body mass?

For demo purposes, we'll drop all columns except the variables whose relationships we're interested in modeling.

In [ ]:
df = df[["bill_depth_mm", "flipper_length_mm", "body_mass_g"]]
df

Suppose we want to create a linear regression model that predicts a penguin's **bill depth** $y$ using both their **flipper length** $x_1$ and **body mass** $x_2$, plus an intercept term.

$$\Large \hat{y} = \theta_0 + \theta_1 x_1 + \theta_2 x_2$$

Another way to write this:

$$ \widehat{\texttt{bill\_depth\_mm}} = \theta_0 + \theta_1 * \texttt{flipper\_length\_mm} + \theta_2 * \texttt{body\_mass\_g} $$

### OLS Approach 1: Use Solution to Normal Equation

We saw in last lecture that we can model the above multiple linear regression equation using matrix multiplication:

$$ \large \hat{\mathbb{Y}} = \mathbb{X} \theta$$

The optimal $\hat{\theta}$ that minimizes MSE also solves the **normal equation**:

$$ \left( \mathbb{X}^T \mathbb{X} \right) \hat{\theta} = \mathbb{X}^T \mathbb{Y}$$

If $\mathbb{X}$ is full column rank, then there is a unique solution to $\hat{\theta}$:

$$\large \hat{\theta} = \left( \mathbb{X}^T \mathbb{X} \right)^{-1} \mathbb{X}^T \mathbb{Y}$$

> Note: This derivation is one of the most challenging in the course, especially for those of you who are learning linear algebra during the same semester. Be kind to yourself! 

<br/>

---

Let's try this in code. We have to add a bias column to the design matrix if we want to include an intercept in the model: 

In [ ]:
# Construct the design matrix X as a dataframe and add a bias column
X = df[["flipper_length_mm", "body_mass_g"]]

# Notice how Pandas automatically broadcasts 1 to all rows.
X["bias"] = 1

# Let's put the bias column in front.
X = X[["bias", "flipper_length_mm", "body_mass_g"]]

X

In [ ]:
# Construct the outcome vector y
y = df["bill_depth_mm"]
y

Recall the solution to the normal equation if $\mathbb{X}$ is full column rank:

$$\hat{\theta} = \left( \mathbb{X}^T \mathbb{X} \right)^{-1} \mathbb{X}^T \mathbb{Y}$$

In [ ]:
# M.T transposes a matrix M
# X @ Y multiplies matrices X and Y
# np.linalg.inv(M) finds the inverse of a matrix M
# By default, the dataframe X will be silently converted to a matrix
theta_using_normal_equation = np.linalg.inv(X.T @ X) @ X.T @ y
theta_using_normal_equation

So, our fitted model is as follows:

$$ \widehat{\texttt{bill\_depth\_mm}} = 11 + 0.0098 * \texttt{flipper\_length\_mm} + 0.0015 * \texttt{body\_mass\_g} $$

Note: The code above is inefficient. We won't go into this in Data 100, but it's better to use `np.linalg.solve` rather than computing the explicit matrix inverse:

In [ ]:
# Solves for θ in (XTX)θ = XTy (i.e., the normal equation) using a system of linear equations
np.linalg.solve(X.T @ X, X.T @ y)

#### Make Predictions
We'll save the predictions in a column so we can compare them against predictions from `sklearn` in the next section.

In [ ]:
# Convert the dataframe representation of the design matrix to a numpy array, 
# and then multiply it by the theta vector to get the predicted outcomes.
# Yhat = Xθ
df["pred_bill_depth_mm"] = X.to_numpy() @ theta_using_normal_equation
df

<br/><br/>

### Using `sklearn` to fit our Multiple Linear Regression Model


An alternate approach to optimize our model is to use the `sklearn.linear_model.LinearRegression` class. [(Documentation)](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LinearRegression.html) 

In [ ]:
# Import the LinearRegression class
from sklearn.linear_model import LinearRegression

1. **Create an `sklearn` object.**

    First we create a model. At this point the model has not been fit so it has no parameters.

In [ ]:
# Construct a blank LinearRegression model object
model = LinearRegression()
model

2. **`fit` the object to data.**

Then we "fit" the model, which means computing the parameters that minimize the loss function. 
    
The first argument of the fit function should be a matrix (or DataFrame), and the second should be the observations we're trying to predict. 

Note: The `LinearRegression` class is hard-coded to use the **MSE** as its loss function. 

In [ ]:
# Fit the linear regression model to the data
model.fit(
    X=df[["flipper_length_mm", "body_mass_g"]], 
    y=df["bill_depth_mm"]
  )

3. **Analyze fit or call `predict`.**

    Now that our model is trained, we can ask it questions. 


The code below asks the model to estimate the bill depth (in mm) for three different penguins with different flipper lengths and body masses.

In [ ]:
# Generate a prediction (yhat) by providing the X values to the model.
# The input to .predict() is a list of lists. 
# Each sub-list contains the X values for each observation.
# That's why there are double brackets!
model.predict([[185, 3750.0], [175, 3550.0], [145, 3250.0]]) 

We can also ask the model to generate a series of predictions:

In [ ]:
# Can also provide a design matrix or dataframe to .predict()
df["sklearn_preds"] = model.predict(df[["flipper_length_mm", "body_mass_g"]])
df

Looking at the predictions above, we see that they are exactly the same as what we calculated with our analytic formula!

**Analyze parameters.**

We can ask the model for its intercept and slope with `_intercept` and `_coef`, respectively.

In [ ]:
# There is one intercept term in the model
model.intercept_

In [ ]:
# There are two slope terms in the model
model.coef_

So, our fitted model is as follows:

$$ \widehat{\texttt{bill\_depth\_mm}} = 11 + 0.0098 * \texttt{flipper\_length\_mm} + 0.0015 * \texttt{body\_mass\_g} $$

The parameters are the same as with our analytic formula.

In [ ]:
# vs. analytical solutions
theta_using_normal_equation

They look the same!

### Visualize the Fit
We can visualize this data in three dimensions, but for many (most) real-world problems this will not be possible (or helpful).

Note, the following code is out of scope for this class.

In [ ]:
fig = px.scatter_3d(df, x="flipper_length_mm", y="body_mass_g", z="bill_depth_mm")

# Create a grid of points to evaluate plane
grid_resolution = 2
(u,v) = np.meshgrid(
    np.linspace(df["flipper_length_mm"].min(), df["flipper_length_mm"].max(), grid_resolution),
    np.linspace(df["body_mass_g"].min(), df["body_mass_g"].max(), grid_resolution))
features = pd.DataFrame({"flipper_length_mm": u.flatten(),
                         "body_mass_g": v.flatten()})
# Make predictions at every point on the grid
zs = model.predict(features)

# create the Surface
fig.add_trace(go.Surface(x=u, y=v, z= zs.reshape(u.shape), opacity=0.9, showscale=False))
fig.update_layout(autosize=False, width=800, height=600)

We see that the predictions all lie in a plane. In higher dimensions, the predictions all lie in a "hyperplane". 

**Analyze performance.**

The `sklearn` package also provides a function `mean_squared_error()` ([documentation](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.mean_squared_error.html)) that computes the MSE from a list of observations and predictions. This avoids us having to manually compute MSE by first computing residuals.



In [ ]:
# Calculate the MSE for a prediction vector and a ground truth outcome vector
from sklearn.metrics import mean_squared_error

mean_squared_error(df["bill_depth_mm"], df["sklearn_preds"])

In [ ]:
print(np.min(df["bill_depth_mm"]), np.max(df["bill_depth_mm"]))